## СОЕДИНЕНИЕ СТРОК (КОНКАТЕНАЦИЯ)

Для начала познакомимся с оператором конкатенации строк — || (две вертикальные черты). Он позволяет объединять две и более строки.

Конструкции с оператором соединения строк записываются следующим образом:

строка1 || строка2 || ... || строкаN

``` sql

select 'select * from ' || t.table_schema || '.' || t.table_name || ';' query
from information_schema.tables t
where table_schema = 'shipping'

-- В результате должно получиться пять SQL-запросов, по одному к каждой таблице из схемы shipping.

-- query
-- select * from shipping.truck;
-- select * from shipping.city;
-- select * from shipping.customer;
-- select * from shipping.driver;
-- select * from shipping.shipment;

-- Как мы видим, соединять можно и рукописный текст, и значения столбцов в любом произвольном порядке.

select 'Мама ' || 'мыла ' || 'раму ' || '!'

-- Мама мыла раму !
```

Примечание: Если вы соединяете любую строку и NULL, то результатом будет NULL. Поэтому, если вы формируете какой-то текст на основе поля, в котором присутствует NULL, используйте оператор **coalesce**.

``` sql
-- Ваш заказ доставит водитель #Имя Фамилия#. Его контактный номер: #Номер#
-- Где #Имя Фамилия# и #Номер# взяты из справочника водителей. Если номер не указан, то выведите прочерк (-). Для номеров рекомендуем использовать COALESCE. 
-- Пример из таблицы для наглядности:
-- Ваш заказ доставит водитель Adel Al-Alawi. Его контактный номер: (901) 947-4433
-- Столбец к выдаче — msg (текст сообщения).

-- Первый вариант
select 
'Ваш заказ доставит водитель '||d.first_name||' '||d.last_name||'. '||'Его контактный номер: '||coalesce(d.phone, '-') msg
from
sql.driver d


-- Второй вариант
select
concat(
    'Ваш заказ доставит водитель ', 
    d.first_name, 
    ' ',
    d.last_name, 
    '. ', 
    'Его контактный номер: ', 
    coalesce(d.phone, '-')
) msg
from
sql.driver d
```

## UPPER() И LOWER()

Функции upper(your_text) и lower(your_text) переводят каждый символ вашего текста в верхний и нижний регистр соответственно.

``` sql

select upper('alexander the best!') upper_str, lower('alexander the best!') lower_str

```

Чаще всего эти функции используются для унификации и стандартизации, особенно они актуальны для данных, введённых вручную.

Например, названия города в анкете можно написать разными способами, но символьный состав останется одним и тем же (Москва, москва, МОСКВА).

Результат функций upper() и lower() — тоже строковый, а значит, к нему можно применять все функции, применимые к этому типу данных.

``` sql

-- Запрос, который выводит все id названий клиентов, у которых более десяти доставок, в нижнем регистре. 
-- Отсортируйте результат по cust_id в порядке возрастания. 
-- Столбцы в выдаче: cust_id (id клиента) и cust_name (название клиента в нижнем регистре).

select
c.cust_id,
lower(c.cust_name)
from
sql.customer c
join sql.shipment s on c.cust_id = s.cust_id
group by c.cust_id, c.cust_name
having count(*) > 10
order by c.cust_id
```

## REPLACE()

С помощью функции replace() можно заменять символы в строках.

``` sql

-- Результат выполнения такого запроса будет молоко, т. е. все буквы «а» в строке «малако» были заменены на «о»

select replace('малако', 'а', 'о') -- replace('где', 'что', 'на что')

-- Еще пример где мы заменяем слово машина на слово матрас

select replace('машина', 'шина', 'трас') -- В строке 'машина' мы заменили строку 'шина' на строку 'трас'.


-- Составим справочник utm-меток, для того чтобы передавать город и штат прямо в адресной строке. 
--  Напишите SQL-запрос, который выведет список сочетаний из справочника следующего вида: 
--  название_штата__название_города, где названия штата и города взяты из справочника городов и переведены в нижний регистр. 
--  Столбец к выдаче — utm (форматированный штат-город). Отсортируйте полученный справочник по алфавиту. Обратите внимание! 
--  Все пробелы в названиях городов и штатов замените символом '_' (одно нижнее подчёркивание), а для разделения названий города и штата используйте '__' 
--  (два последовательных нижних подчёркивания). Пример из таблицы для наглядности: new_jersey__union_city

select
lower(replace(c.state, ' ', '_')) || '__' || lower(replace(c.city_name, ' ', '_')) utm
from
sql.city c
order by 1
```

## LEFT() И RIGHT()

Теперь познакомимся с функциями, обрезающими строки.

``` sql
-- Данные функции оставляют количество символов в строке т.е left('абвгде', 3) оставляет 3 cимвола слева; right('абвгде', 3) оставляем 3 символа справа.

select left('абвгде', 3), right('абвгде', 3) 

-- Если использовать отрицательные значения, то все происходит равным счетом наоборот, т.е где отрицательное значение мы наоборот убираем.

select left('абвгде', -3), right('абвгде', -2), left('абвгде', -2), right('абвгде', -2) 

```

Функции left(string,n) и right(string,n) оставляют n левых или правых символов от строки, поданной на вход. Давайте разобьём строку 'Один два три' на слова, используя эти функции.

``` sql

with t as (
select 'Один два три'::text sample_string
)
select 
left(t.sample_string, 3) one,
right(left(t.sample_string, 8), 3) two,
right(t.sample_string, 3) three
from t

-- Результат: 01234567 и 23456789 (в первом случае — восемь символов с «отрезанными» 89 и во втором случае — восемь символов с «отрезанными» 01)

select left('0123456789', - 2), right('0123456789', - 2)

-- SQL-запрос, который выведет первые четыре символа названия штата и количество уникальных названий штатов, которому они соответствуют. 
-- Оставьте только те, которые относятся к двум и более штатам. Добавьте сортировку по первому столбцу. 
-- Столбцы в выдаче: code (четыре первых символа в названии штата), 
-- qty (количество уникальных названий штата, начинающихся с этих символов).

select
distinct
left(c.state, 4) code,
count(distinct c.state) qty
from
sql.city c
group by left(c.state, 4)
having count(distinct c.state) >= 2
order by 1

```



## FORMAT()

Функция format() используется для составления форматированного текста с подстановками. То же самое можно сделать через конкатенацию строк, но это неудобно и громоздко.

Синтаксис функции выглядит следующим образом:
``` sql
format(formatstr text [, argument1 text,argument2 text...])
```
где formatstr — это шаблон, который мы передаём. Это обычная строка, в которой указаны места для подстановки аргумента.

``` sql
-- FORMAT()
select format('Hello, %s %s!', d.first_name, d.last_name)
from sql.driver d

-- Чтобы оставить ковычку в тексте
select $$ some string with quotes ' $$
```

``` sql
-- SQL-запрос, который выведет описание региона в следующем формате:
-- [city_name] is located in [state]. There's [population] people living there. Its area is [area]
-- Обратите внимание, точку в конце ставить не нужно. 
-- Отсортируйте по названию города в алфавитном порядке. Столбец к выдаче — str (сводка).
-- Пример:
-- Abilene is located in Texas. There's 115930 people living there. Its area is 105.10

select
format(
    $$[%s] is located in [%s]. There's [%s] people living there. Its area is [%s]$$, 
    c.city_name, 
    c.state, 
    c.population, 
    c.area
    ) str
from
sql.city c
order by 1
```